In [17]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

In [18]:

from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.lof import LOF
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.hbos import HBOS
from pyod.models.abod import ABOD
from pyod.models.iforest import IForest
from pyod.models.feature_bagging import FeatureBagging


In [19]:
from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

In [20]:
from glob import glob, iglob
mat_file_list = [i for i in glob('.\Anamoly_detec_data\*.mat')]
mat_file_list

['.\\Anamoly_detec_data\\arrhythmia.mat',
 '.\\Anamoly_detec_data\\cardio.mat',
 '.\\Anamoly_detec_data\\glass.mat',
 '.\\Anamoly_detec_data\\ionosphere.mat',
 '.\\Anamoly_detec_data\\letter.mat',
 '.\\Anamoly_detec_data\\lympho.mat',
 '.\\Anamoly_detec_data\\mnist.mat',
 '.\\Anamoly_detec_data\\musk.mat',
 '.\\Anamoly_detec_data\\optdigits.mat',
 '.\\Anamoly_detec_data\\pendigits.mat',
 '.\\Anamoly_detec_data\\pima.mat',
 '.\\Anamoly_detec_data\\satellite.mat',
 '.\\Anamoly_detec_data\\satimage-2.mat',
 '.\\Anamoly_detec_data\\shuttle.mat',
 '.\\Anamoly_detec_data\\vertebral.mat',
 '.\\Anamoly_detec_data\\vowels.mat',
 '.\\Anamoly_detec_data\\wbc.mat']

In [21]:
df_columns=['Data','#Sample','#Dimensions','Outlier Perc','PCA','MCD','OCSVM','LOF','CBLOF','KNN','HBOS','ABOD','IFOREST','FEATUREBAGGING']

In [22]:
roc_df = prn_df = time_df = pd.DataFrame(columns=df_columns)
print(roc_df,prn_df,time_df)

Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, PCA, MCD, OCSVM, LOF, CBLOF, KNN, HBOS, ABOD, IFOREST, FEATUREBAGGING]
Index: [] Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, PCA, MCD, OCSVM, LOF, CBLOF, KNN, HBOS, ABOD, IFOREST, FEATUREBAGGING]
Index: [] Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, PCA, MCD, OCSVM, LOF, CBLOF, KNN, HBOS, ABOD, IFOREST, FEATUREBAGGING]
Index: []


In [23]:
from time import time
random_state = np.random.RandomState(42)

for mat_file in mat_file_list:
        print(f'\n... Processing {mat_file}...')
        mat = loadmat(mat_file)

        X = mat['X']
        y = mat['y'].ravel()
        outliers_fraction = np.count_nonzero(y) / len(y)
        outliers_percentage = round(outliers_fraction * 100, ndigits=4)

        # construct containers for saving results
        roc_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
        prn_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
        time_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]

        # 60% data for training and 40% for testing
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = random_state)

        # standardizing data for processing
        X_train_norm, X_test_norm = standardizer(X_train, X_test)
        
        classifiers = {'Angle-based Outlier Detector (ABOD)': ABOD(contamination=outliers_fraction),
                       
                       'Cluster-based Local Outlier Factor': CBLOF(contamination=outliers_fraction, check_estimator=False,
                                                               random_state=random_state),
                       
                       'Feature Bagging': FeatureBagging(contamination=outliers_fraction, random_state=random_state),
                       
                       'Histogram-base Outlier Detection (HBOS)': HBOS(contamination=outliers_fraction),
                       
                       'Isolation Forest': IForest(contamination=outliers_fraction, random_state=random_state),
                       
                       'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
                       
                       'Local Outlier Factor (LOF)': LOF(contamination=outliers_fraction),
                        'Minimum Covariance Determinant (MCD)': MCD(contamination=outliers_fraction, random_state=random_state),
                       
                       'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction),
                       
                       'Principal Component Analysis (PCA)': PCA(contamination=outliers_fraction, random_state=random_state)
                      }
        
        for clf_name, clf in classifiers.items():
            t0 = time()
            
            clf.fit(X_train_norm)
            
            test_scores = clf.decision_function(X_test_norm)
            t1 = time()
            
            duration = round(t1 - t0, ndigits=4)
            time_list.append(duration)
            
            roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
            
            prn = round(precision_n_scores(y_test, test_scores), ndigits=4)
            
            print(f'{clf_name} \n\tROC: {roc} \tprecision @ rank n: {prn} \texecution time: {duration}s')
            
            roc_list.append(roc)
            prn_list.append(prn)
            
        temp_df = pd.DataFrame(time_list).transpose()
        temp_df.columns = df_columns
        time_df = pd.concat([time_df, temp_df], axis=0)
        
        temp_df = pd.DataFrame(roc_list).transpose()
        temp_df.columns = df_columns
        roc_df = pd.concat([roc_df, temp_df], axis=0)
        
        temp_df = pd.DataFrame(prn_list).transpose()
        temp_df.columns = df_columns
        prn_df = pd.concat([prn_df, temp_df], axis=0)
        
        print('\n****************************************')
                                              

                                    



... Processing .\Anamoly_detec_data\arrhythmia.mat...
Angle-based Outlier Detector (ABOD) 
	ROC: 0.7687 	precision @ rank n: 0.3571 	execution time: 0.261s
Cluster-based Local Outlier Factor 
	ROC: 0.7684 	precision @ rank n: 0.4643 	execution time: 0.166s
Feature Bagging 
	ROC: 0.7799 	precision @ rank n: 0.5 	execution time: 0.695s
Histogram-base Outlier Detection (HBOS) 
	ROC: 0.8511 	precision @ rank n: 0.5714 	execution time: 0.131s
Isolation Forest 
	ROC: 0.8527 	precision @ rank n: 0.5714 	execution time: 0.759s
K Nearest Neighbors (KNN) 
	ROC: 0.782 	precision @ rank n: 0.5 	execution time: 0.098s
Local Outlier Factor (LOF) 
	ROC: 0.7787 	precision @ rank n: 0.4643 	execution time: 0.071s


C:\Users\user\anaconda\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


Minimum Covariance Determinant (MCD) 
	ROC: 0.8228 	precision @ rank n: 0.4286 	execution time: 1.2609s
One-class SVM (OCSVM) 
	ROC: 0.7986 	precision @ rank n: 0.5 	execution time: 0.04s
Principal Component Analysis (PCA) 
	ROC: 0.7997 	precision @ rank n: 0.5 	execution time: 0.076s

****************************************

... Processing .\Anamoly_detec_data\cardio.mat...
Angle-based Outlier Detector (ABOD) 
	ROC: 0.5763 	precision @ rank n: 0.1875 	execution time: 0.71s
Cluster-based Local Outlier Factor 
	ROC: 0.8221 	precision @ rank n: 0.4844 	execution time: 0.249s
Feature Bagging 
	ROC: 0.4879 	precision @ rank n: 0.1406 	execution time: 0.966s
Histogram-base Outlier Detection (HBOS) 
	ROC: 0.8453 	precision @ rank n: 0.4688 	execution time: 0.01s
Isolation Forest 
	ROC: 0.9414 	precision @ rank n: 0.5 	execution time: 0.5409s
K Nearest Neighbors (KNN) 
	ROC: 0.6959 	precision @ rank n: 0.2812 	execution time: 0.221s
Local Outlier Factor (LOF) 
	ROC: 0.4715 	precision @ rank 

C:\Users\user\anaconda\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


Minimum Covariance Determinant (MCD) 
	ROC: 0.8781 	precision @ rank n: 0.3906 	execution time: 0.7529s
One-class SVM (OCSVM) 
	ROC: 0.9507 	precision @ rank n: 0.5938 	execution time: 0.101s
Principal Component Analysis (PCA) 
	ROC: 0.9638 	precision @ rank n: 0.6875 	execution time: 0.005s

****************************************

... Processing .\Anamoly_detec_data\glass.mat...
Angle-based Outlier Detector (ABOD) 
	ROC: 0.7104 	precision @ rank n: 0.25 	execution time: 0.079s
Cluster-based Local Outlier Factor 
	ROC: 0.8506 	precision @ rank n: 0.25 	execution time: 0.078s
Feature Bagging 
	ROC: 0.7043 	precision @ rank n: 0.25 	execution time: 0.046s
Histogram-base Outlier Detection (HBOS) 
	ROC: 0.6524 	precision @ rank n: 0.0 	execution time: 0.004s
Isolation Forest 
	ROC: 0.7195 	precision @ rank n: 0.25 	execution time: 0.397s
K Nearest Neighbors (KNN) 
	ROC: 0.7805 	precision @ rank n: 0.25 	execution time: 0.016s
Local Outlier Factor (LOF) 
	ROC: 0.7774 	precision @ rank n: 

...

Minimum Covariance Determinant (MCD) 
	ROC: 0.863 	precision @ rank n: 0.3973 	execution time: 5.3137s
One-class SVM (OCSVM) 
	ROC: 0.8417 	precision @ rank n: 0.3801 	execution time: 5.1948s
Principal Component Analysis (PCA) 
	ROC: 0.8396 	precision @ rank n: 0.3767 	execution time: 0.189s

****************************************

... Processing .\Anamoly_detec_data\musk.mat...
Angle-based Outlier Detector (ABOD) 
	ROC: 0.0809 	precision @ rank n: 0.0333 	execution time: 3.4688s
Cluster-based Local Outlier Factor 
	ROC: 1.0 	precision @ rank n: 1.0 	execution time: 0.6529s
Feature Bagging 
	ROC: 0.5228 	precision @ rank n: 0.1667 	execution time: 14.2163s
Histogram-base Outlier Detection (HBOS) 
	ROC: 0.9999 	precision @ rank n: 0.9667 	execution time: 0.1s
Isolation Forest 
	ROC: 0.9996 	precision @ rank n: 0.9333 	execution time: 2.3189s
K Nearest Neighbors (KNN) 
	ROC: 0.7348 	precision @ rank n: 0.2333 	execution time: 2.4869s
Local Outlier Factor (LOF) 
	ROC: 0.5323 	precision 

C:\Users\user\anaconda\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


Minimum Covariance Determinant (MCD) 
	ROC: 0.3486 	precision @ rank n: 0.0 	execution time: 2.2829s
One-class SVM (OCSVM) 
	ROC: 0.4972 	precision @ rank n: 0.0 	execution time: 1.5799s
Principal Component Analysis (PCA) 
	ROC: 0.504 	precision @ rank n: 0.0 	execution time: 0.068s

****************************************

... Processing .\Anamoly_detec_data\pendigits.mat...
Angle-based Outlier Detector (ABOD) 
	ROC: 0.7008 	precision @ rank n: 0.0308 	execution time: 3.1488s
Cluster-based Local Outlier Factor 
	ROC: 0.9609 	precision @ rank n: 0.3077 	execution time: 0.4519s
Feature Bagging 
	ROC: 0.4687 	precision @ rank n: 0.0462 	execution time: 6.5837s
Histogram-base Outlier Detection (HBOS) 
	ROC: 0.9294 	precision @ rank n: 0.2615 	execution time: 0.015s
Isolation Forest 
	ROC: 0.9422 	precision @ rank n: 0.2769 	execution time: 1.071s
K Nearest Neighbors (KNN) 
	ROC: 0.7602 	precision @ rank n: 0.0462 	execution time: 0.952s
Local Outlier Factor (LOF) 
	ROC: 0.481 	precision 

C:\Users\user\anaconda\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-50.337315560576357 > -78.193487063766270). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
C:\Users\user\anaconda\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-50.088727288180465 > -77.017915011470095). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
C:\Users\user\anaconda\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-50.582785491594180 > -76.896496195181470). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
C:\Users\u

Minimum Covariance Determinant (MCD) 
	ROC: 0.9903 	precision @ rank n: 0.7534 	execution time: 18.417s
One-class SVM (OCSVM) 
	ROC: 0.9922 	precision @ rank n: 0.9553 	execution time: 76.6131s
Principal Component Analysis (PCA) 
	ROC: 0.9902 	precision @ rank n: 0.9503 	execution time: 0.053s

****************************************

... Processing .\Anamoly_detec_data\vertebral.mat...
Angle-based Outlier Detector (ABOD) 
	ROC: 0.2797 	precision @ rank n: 0.0 	execution time: 0.101s
Cluster-based Local Outlier Factor 
	ROC: 0.3908 	precision @ rank n: 0.0 	execution time: 0.077s
Feature Bagging 
	ROC: 0.3027 	precision @ rank n: 0.0 	execution time: 0.047s
Histogram-base Outlier Detection (HBOS) 
	ROC: 0.2695 	precision @ rank n: 0.0 	execution time: 0.004s
Isolation Forest 
	ROC: 0.3576 	precision @ rank n: 0.0 	execution time: 0.402s
K Nearest Neighbors (KNN) 
	ROC: 0.318 	precision @ rank n: 0.0 	execution time: 0.016s
Local Outlier Factor (LOF) 
	ROC: 0.318 	precision @ rank n: 0

In [27]:
roc_df

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING
0,.\Anamoly_detec_data\arrhythmia,452,274,14.6018,0.7687,0.7684,0.7799,0.8511,0.8527,0.782,0.7787,0.8228,0.7986,0.7997
0,.\Anamoly_detec_data\cardio,1831,21,9.6122,0.5763,0.8221,0.4879,0.8453,0.9414,0.6959,0.4715,0.8781,0.9507,0.9638
0,.\Anamoly_detec_data\glass,214,9,4.2056,0.7104,0.8506,0.7043,0.6524,0.7195,0.7805,0.7774,0.7165,0.6189,0.622
0,.\Anamoly_detec_data\ionosphere,351,33,35.8974,0.9004,0.8952,0.8933,0.5195,0.8309,0.9134,0.8989,0.9399,0.8372,0.7971
0,.\Anamoly_detec_data\letter,1600,32,6.25,0.8465,0.7423,0.866,0.5728,0.5778,0.845,0.8409,0.7499,0.5744,0.48
0,.\Anamoly_detec_data\lympho,148,18,4.0541,0.9382,0.9709,0.9673,0.9964,0.9855,0.9636,0.9636,0.9164,0.9636,0.9818
0,.\Anamoly_detec_data\mnist,7603,100,9.2069,0.7813,0.8447,0.7259,0.5675,0.7801,0.8409,0.7085,0.863,0.8417,0.8396
0,.\Anamoly_detec_data\musk,3062,166,3.1679,0.0809,1,0.5228,0.9999,0.9996,0.7348,0.5323,1,1,1
0,.\Anamoly_detec_data\optdigits,5216,64,2.8758,0.4428,0.7852,0.4641,0.8822,0.5764,0.3824,0.4584,0.3486,0.4972,0.504
0,.\Anamoly_detec_data\pendigits,6870,16,2.2707,0.7008,0.9609,0.4687,0.9294,0.9422,0.7602,0.481,0.8271,0.93,0.9332


In [28]:
prn_df

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING
0,.\Anamoly_detec_data\arrhythmia,452,274,14.6018,0.3571,0.4643,0.5,0.5714,0.5714,0.5,0.4643,0.4286,0.5,0.5
0,.\Anamoly_detec_data\cardio,1831,21,9.6122,0.1875,0.4844,0.1406,0.4688,0.5,0.2812,0.125,0.3906,0.5938,0.6875
0,.\Anamoly_detec_data\glass,214,9,4.2056,0.25,0.25,0.25,0,0.25,0.25,0.25,0,0.25,0.25
0,.\Anamoly_detec_data\ionosphere,351,33,35.8974,0.8214,0.8036,0.75,0.3393,0.6607,0.8393,0.75,0.8571,0.7143,0.5893
0,.\Anamoly_detec_data\letter,1600,32,6.25,0.275,0.175,0.4,0.125,0.05,0.3,0.325,0.075,0.1,0.05
0,.\Anamoly_detec_data\lympho,148,18,4.0541,0.4,0.6,0.6,0.8,0.6,0.6,0.6,0.6,0.6,0.8
0,.\Anamoly_detec_data\mnist,7603,100,9.2069,0.3562,0.4007,0.3664,0.1199,0.2979,0.4144,0.339,0.3973,0.3801,0.3767
0,.\Anamoly_detec_data\musk,3062,166,3.1679,0.0333,1,0.1667,0.9667,0.9333,0.2333,0.1333,0.9667,1,1
0,.\Anamoly_detec_data\optdigits,5216,64,2.8758,0.0161,0,0.0484,0.2581,0.0161,0,0.0484,0,0,0
0,.\Anamoly_detec_data\pendigits,6870,16,2.2707,0.0308,0.3077,0.0462,0.2615,0.2769,0.0462,0.0462,0.0615,0.2923,0.3385


In [29]:
time_df

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING
0,.\Anamoly_detec_data\arrhythmia,452,274,14.6018,0.261,0.166,0.695,0.131,0.759,0.098,0.071,1.2609,0.04,0.076
0,.\Anamoly_detec_data\cardio,1831,21,9.6122,0.71,0.249,0.966,0.01,0.5409,0.221,0.114,0.7529,0.101,0.005
0,.\Anamoly_detec_data\glass,214,9,4.2056,0.079,0.078,0.046,0.004,0.397,0.016,0.004,0.052,0.003,0.003
0,.\Anamoly_detec_data\ionosphere,351,33,35.8974,0.15,0.072,0.084,0.014,0.429,0.028,0.008,0.094,0.006,0.004
0,.\Anamoly_detec_data\letter,1600,32,6.25,0.642,0.1709,0.851,0.016,0.559,0.196,0.106,1.5409,0.101,0.007
0,.\Anamoly_detec_data\lympho,148,18,4.0541,0.057,0.075,0.042,0.008,0.39,0.011,0.003,0.058,0.003,0.003
0,.\Anamoly_detec_data\mnist,7603,100,9.2069,11.1394,1.9219,66.6085,0.084,3.5528,10.3155,9.5955,5.3137,5.1948,0.189
0,.\Anamoly_detec_data\musk,3062,166,3.1679,3.4688,0.6529,14.2163,0.1,2.3189,2.4869,1.8139,23.0598,1.2369,0.211
0,.\Anamoly_detec_data\optdigits,5216,64,2.8758,3.7708,0.777,15.3712,0.053,1.8239,2.2489,1.7659,2.2829,1.5799,0.068
0,.\Anamoly_detec_data\pendigits,6870,16,2.2707,3.1488,0.4519,6.5837,0.015,1.071,0.952,0.8139,3.1898,1.3979,0.0157
